In [ ]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=0)

In [ ]:
%%time
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math

from helper import get_dte, expPricePct, get_prec, grp_opts, short_opt_margin, filter_kxdte, strat_hilo52, strat_onlyputs, get_div_tick, catch, get_onlyputs

#...assignments
#______________

fspath = '../data/nse/'

m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.05       # Upper or Lower base multiple for prices
expmult = 1.1     # Used to set higher expPrice for expPrice = price

desired_rom = 0.8 # desired rom to give the target price.

min_rom = 0.2
min_pop = 0.85
min_dte = 45     # no of minimum dte days to determine ohlc filter for min and max

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin
    
#...get current positions
#________________________

#... read the account info
ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...Harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
strike = [d.strike for d in df_p.contract]

df_p['symbol'] = [s.symbol for s in df_p.contract.values]
df_p['expiry'] = expiry
df_p['strike'] = strike

df_p = df_p.assign(type=[type(x).__name__ for x in df_p.contract])

# harvest option Dataframe
df_hop = df_p[df_p.type == 'Option'].reset_index(drop=True)

df_hop['dte'] = [get_dte(d.lastTradeDateOrContractMonth) for d in df_hop.contract]

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice
discount = [m for m in map(expPricePct, df_hop.expiry)]

df_hop['hvstPrice'] = pd.concat([df_hop.averageCost*discount, 
                               df_hop.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_hop.hvstPrice = np.floor(df_hop.hvstPrice/base)*base # round down to the nearest 0.05

df_hop.loc[df_hop.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
df_hop['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_hop.position, df_hop.hvstPrice)]

# ignore data for dte < 3 days. These are as good as gone.
df_h = df_hop[df_hop.dte > 0].reset_index(drop=True)

hqc = ib.qualifyContracts(*df_h.contract)
df_h = df_h.assign(qual_contract=hqc)

# ...sowing prepration
#_____________________
 #... find margins
orders = [Order(action=np.where(q>0, 'SELL', 'BUY'), totalQuantity=abs(q), orderType='MKT') for q in df_p.position]

cs = list(df_p.contract)
qc = ib.qualifyContracts(*cs)
df_p = df_p.assign(margin=[float(ib.whatIfOrder(c, o).initMarginChange) for c, o in zip(qc, orders)])

#... get the lots
tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

# Truncate to 9 characters for ibSymbol
df_tp['symbol'] = df_tp.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_tp.symbol = df_tp.symbol.replace(ntoi)

pos_cols = ['symbol', 'type', 'position', 'marketValue', 'unrealizedPNL', 'realizedPNL', 'margin']
df_pos = df_p[pos_cols]

#... make the blacklist
df1 = df_pos.groupby('symbol').sum()

df1 = df1.join(df_tp[['symbol', 'lot']].set_index('symbol'), how='inner')

df1['marginUsed'] =  df1.position/df1.lot*df1.margin

df1['marginAvlbl'] = df1.marginUsed + max_p

df1['qtyAvlbl'] = np.where(df1.marginAvlbl > df1.margin, np.floor(df1.marginAvlbl/df1.margin), 0)

blacklist = list(df1[df1.qtyAvlbl <= 0].index)
remqtydict  = df1[~df1.index.isin(blacklist)].qtyAvlbl.to_dict() # remaining quantity not in blacklist

#...build the high-pop-roc dataframe
fs = listdir(fspath)

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

df_opt = pd.concat([pd.read_pickle(fspath+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle(fspath+f).reset_index() for f in ohlcs], axis=0, sort=True)
df_und = pd.concat([pd.read_pickle(fspath+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# arrange the columns
cols = ['symbol', 'right', 'expiry', 'dte', 'strike', 'undPrice', 'lo52',  'hi52', 
'stdev', 'volatility', 'margin', 'lot', 'bsmPrice', 'pop', 'rom', 'price', 'option']

df_opt1 = df_opt[cols]

# take only high pops
df_opt2 = df_opt1[df_opt1['pop'] >= min_pop]

 # Limit the number of options
df_opt3 = strat_hilo52(df_opt2) # breaking highs and lows
df_opt3 = df_opt3.sort_values('rom', ascending = False)

df_opt4 = df_opt3[(df_opt3.price > 0) & (df_opt3.rom > min_rom)]

#...Get the spread and underlying
contracts = list(df_opt4.option)
for contract in contracts:
    ib.reqMktData(contract, '', False, False)
ib.sleep(5)
for contract in contracts:
    ib.cancelMktData(contract)

tickers = [catch(lambda: ib.ticker(ticker)) for ticker in contracts]

asks = [catch(lambda: t.ask) for t in tickers]
bids = [catch(lambda: t.bid) for t in tickers]
close = [catch(lambda: t.close) for t in tickers]
undPrices = [catch(lambda: t.modelGreeks.undPrice) for t in tickers]

df_opt4 = df_opt4.assign(ask=asks, bid=bids, close=close, undPrice=undPrices)

# make ask price to be 10% more than close for ask < 0
df_opt4.loc[df_opt4.ask < 0, 'ask'] = 1.1 * df_opt4.close

# get the better of price, bsmPrice and spread for the option - with an expected multiple (expmult)
max_price = pd.concat([df_opt4.price*expmult, df_opt4.bsmPrice, df_opt4.close + (df_opt4.ask - df_opt4.close)*0.5], axis=1).max(axis=1)
df_opt4['expPrice'] = get_prec(max_price, base)

# recalculate rom based on expPrice
df_opt4 = df_opt4.assign(romExp=df_opt4.expPrice/df_opt4.margin*365/df_opt4.dte*df_opt4.lot)

# Make df1 the dataframe that you want to execute on!
df1 = df_opt4.copy()   # make this the last dataframe to get the orders placed

df1 = df1.assign(margin_roll=df1.margin.expanding(1).sum())
# df1 = df1[df1.margin_roll < av_funds*2]  # to a max of 2 times available funds

df1 = df1.assign(qty=1) # defaults to a quantity of 1

df1.loc[df1.expPrice <= 0.1, 'expPrice'] = 0.15  # Make the selling price a minimum of 0.15

df = grp_opts(df1) # group by puts and calls in the order going away from the threshold

df = get_onlyputs(df) # get only puts

# Get sort order based on highest rom. The top ones could be the most risky!
sortlist = list(df.sort_values('romExp', ascending=False).groupby('symbol').head(1).symbol)
df = df.set_index('symbol')

df = df.loc[sortlist].reset_index()

contracts = [c for c in df.option]

In [ ]:
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(contracts), \
      len(df.symbol.unique()), sum(df.margin*df.qty), av_funds*max_nlvp, sum(df.expPrice*df.lot*df.qty)))

In [ ]:
# ...review calls and puts

df['remqty'] = round(max_p/df.margin)

cols = ['right', 'symbol', 'strike', 'undPrice', 'dte', 'pop', 'rom', 'price', 'expPrice', 'margin', 'lot', 'qty', 'remqty', 'option']
df = df[cols]

# replace remqty with non-blacklist remaining quantities
df = df.set_index('symbol')
df.remqty = np.where(df.index.isin(remqtydict.keys()), df.index.map(remqtydict), df.remqty)
df = df.reset_index()

# make calls and puts watch - to quickly weed out risky options
gb = df.groupby('right')

if 'C' in [k for k in gb.indices]:
#     df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    df_calls = gb.get_group('C').reset_index(drop=True)
    
    df_callsymbols = df_calls.symbol.unique()
    watchcalls = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df_callsymbols]
    df_wp = util.df(watchcalls)
    df_wp.to_csv(fspath+'callswatch.csv', index=None, header=False)
    
else:
    df_calls = pd.DataFrame([]) # empty dataframe

if 'P' in [k for k in gb.indices]:
#     df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    df_puts = gb.get_group('P').reset_index(drop=True)
    
    # make watchlist
    df_putssymbols = df_puts.symbol.unique()
    watchputs = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df_putssymbols]
    df_wp = util.df(watchputs)
    df_wp.to_csv(fspath+'putswatch.csv', index=None, header=False)
else:
    df_puts = pd.DataFrame([]) # empty dataframe

# output the consolidated puts and calls dataframe
df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
df.to_csv(fspath+'check.csv', index=None, header=True)

# .... for puts from path: C:\Users\kashir\Documents\IBKR\data\nse\putswatch.csv
# .... or in this path: C:\Users\User\Documents\ibkr\nse\zdata\putswatch.csv (home laptop)

In [ ]:
# After going through checked.csv, with puts and calls, eliminate risky options
# Save the file as checked.csv

df_final = pd.read_csv(fspath+'checked.csv') # picks up the checked and ready-to-go contracts
cs = [eval(c) for c in df_final.option]  # convert the "quoted strings" from csv back to object
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df_final.lot, df_final.expPrice)]
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(cs), \
      len(df_final.symbol.unique()), sum(df_final.margin*df_final.qty), av_funds*max_nlvp, sum(df_final.expPrice*df_final.lot*df_final.qty)))